#Prepare Environment

### Install Packages

In [ ]:
!pip install --upgrade geopandas
!pip install --upgrade pyshp
!pip install --upgrade geemap
!pip install --upgrade osgeo

     |████████████████████████████████| 972kB 2.7MB/s 
     |████████████████████████████████| 14.8MB 293kB/s 
     |████████████████████████████████| 10.9MB 35.9MB/s 
     |████████████████████████████████| 225kB 2.8MB/s 
  Created wheel for pyshp: filename=pyshp-2.1.2-cp36-none-any.whl size=36216 sha256=3776ca71c50c2d320cced689750f1aece797d498a1084492af2a949ab7c6f61a
  Stored in directory: /root/.cache/pip/wheels/96/6c/53/4112475adf3b831da97f083163d0f38ee6daac9c1b13f7afea
Successfully built pyshp
     |████████████████████████████████| 378kB 2.7MB/s 
     |████████████████████████████████| 102kB 6.1MB/s 
     |████████████████████████████████| 1.1MB 8.6MB/s 
     |████████████████████████████████| 81kB 7.0MB/s 
     |████████████████████████████████| 102kB 7.2MB/s 
     |████████████████████████████████| 5.9MB 8.8MB/s 
     |████████████████████████████████| 5.1MB 32.5MB/s 
     |████████████████████████████████| 4.7MB 36.6MB/s 
     |████████████████████████████████| 122kB 45.0MB/s 

ERROR: Could not find a version that satisfies the requirement osgeo (from versions: none)
ERROR: No matching distribution found for osgeo


### Load Packages 

In [ ]:
import folium
#from geemap import eefolium as emap
import pandas as pd
import geemap as emap
from osgeo import ogr, osr
import geopandas as gpd

In [ ]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=nAyRuUwbgxBOXZ6ahGSUiYt3F_5GVG7Z_7Oe4s5R_rE&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/4AGxkOXMjthz9AFYlPNx0liOZ0q-gXjEo1_jwYP6N-l2S0AV6Ti1EDM

Successfully saved authorization token.


### Set working directory 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
4/4AGITRYDdsDCl0q6l2e_fNVCvtPxp2dOmuuBoVvxFlO5beudyxVCipQ
Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/Shared drives/APHIS  Projects/eRADS/eRADS_workflow/eRADS_workflow/host_mapping')

# Define Area of Interest (AOI)

# Mapping Crop Distribution using CropLand Data Layer

In [ ]:
aoi = ee.Geometry.Polygon([
    [-76.068, 43.375],
    [-75.181, 43.375],
    [-75.181, 42.005],
    [-76.068, 42.005]
])

#aoi = ee.Projection('EPSG:4326')
centroid = aoi.centroid()
long, lat = centroid.getInfo()['coordinates']
print ('Centroid: long = {}, lat = {}'.format(long, lat))

Centroid: long = -75.62449999999973, lat = 42.68833703924229


## Step 1 -- Extract Crop Distribution from Cropland Data Layer   

Load cropland data layer (cdl) from google earth engine. Please pay attention on the how many years of crop layer data you want to use to drive the crop distribution map.

In [ ]:
cdls = ee.ImageCollection('USDA/NASS/CDL')
clds_dates = cdls.filterDate('2017-01-01', '2020-09-01') # refine data to desired time period

The CDL dataset for each year includes multiple information. To generate the crop map, we only need to select cropland layer from the dataset.

In [ ]:
def select_layer(image):
  return image.select(['cropland'])

crop_layers = clds_dates.map(select_layer)  
print("Band name is: " + str(crop_layers.first().bandNames().getInfo()))
print("Number of layer is: " + str(crop_layers.size().getInfo()))

Band name is: ['cropland']
Number of layer is: 3


## Step 2 -- Conduct a Majority Voting over the the Multiple Years Cropland Data Layers

In [ ]:
crop_mode = crop_layers.reduce(ee.Reducer.mode())
crls_mode_aoi = crop_mode.clip(aoi) # clip to only keep data for aoi, or the data will be for the whole contiguous U.S.

## Step 3 -- Export Crop Distribution Map (Optinoal)

Need to set values to description (name for the crop distribution map),scale (spatial resolution of the exported image), folder (google drive folder where you want to save the map), fileFormat (image format)

In [ ]:
description = 'Crop_Distribution'
scale = 100
folder = 'Host_mapping'
fileFormat = 'GeoTIFF'

The code below exported image to the folder and also report the exporting status every 10 seconds.

In [ ]:
task = ee.batch.Export.image.toDrive(image = crls_mode_aoi.toInt(), maxPixels = 1e13, description = description, scale = scale, folder = folder, fileFormat = fileFormat)
task.start()

import time
starttime=time.time()
state = "Ready"

n=0
while state!='COMPLETED' and state!= 'FAILED':
    
    time.sleep(10.0)
    
    print("status: " + task.status()['state'])
    state = task.status()['state']
    n += 1
    print("Time consumed: " + str(10*n) +'s')
    

if task.status()['state'] == 'FAILED':
    print (task.status()['error_message'])

'READY'

# Mapping Host Using Presence Data of the Host Species and the NAIP Images

## Define AOI and Load NAIP images

### Define AOI, this AOI is defined by the presence points of ECFF

### Read presence points of ECFF in year 2019

In [ ]:
pre19 = gpd.read_file("./PreAbe/2019_Positive_ECFF_Trap_Sites.shp")
print("shape of presence 2019 is", pre19.shape)

shape of presence 2019 is (1115, 41)


### Get min, max of latitude and longitude from the presence points to define AOI for host mapping

In [ ]:
def getCrds_Y(gdf_pts):
    crds_Y = gdf_pts[['Latitude', 'Longitude','Year']]
    return (crds_Y)

crds = getCrds_Y(pre19)

print(crds.shape)
print(crds.head(5))

ctd = crds[['Latitude', 'Longitude']].mean()
print(ctd)

lat_min, lat_max, long_min, long_max = crds['Latitude'].min(), crds['Latitude'].max(), crds['Longitude'].min(), crds['Longitude'].max()
lat_min, lat_max, long_min, long_max = lat_min - 1, lat_max + 1, long_min - 1, long_max + 1

(1115, 3)
    Latitude  Longitude    Year
0  43.362635 -78.568601  2019.0
1  43.362545 -78.568667  2019.0
2  43.363006 -78.567970  2019.0
3  43.362870 -78.568676  2019.0
4  43.362514 -78.569296  2019.0
Latitude     43.287692
Longitude   -78.759667
dtype: float64


### Define AOI based on the point data

In [62]:
def getAOI(pre):
    long_min = pre['Longitude'].min() - 1
    long_max = pre['Longitude'].max() + 1
    lat_min = pre['Latitude'].min() - 1
    lat_max = pre['Latitude'].max() + 1
    
    aoi = ee.Geometry.Polygon([
        [long_min, lat_max],
        [long_max, lat_max],
        [long_max, lat_min],
        [long_min, lat_min]
    ])    
    
    centroid_long = (long_min + long_max)/2
    centroid_lat = (lat_min + lat_max)/2
    
    ctds = [centroid_long, centroid_lat]
    
    return(aoi, ctds)
aoi, ctds = getAOI(crds)

### Load NAIP Images

In [45]:
collection = ee.ImageCollection('USDA/NAIP/DOQQ')
naip = collection.filterBounds(aoi)
naip15 = collection.filterDate('2015-05-01','2015-10-30')
np15 = naip15.mosaic().clip(aoi)
count15 = naip15.size().getInfo()
naip13 = collection.filterDate('2013-03-01','2013-12-30')
naip17 = collection.filterDate('2017-03-01','2017-12-30')
#naip19 = collection.filterDate('2019-03-01','2019-12-30')  # not available yet

np13 = naip13.mosaic().clip(aoi)
np17 = naip17.mosaic().clip(aoi)
#np19 = naip19.mosaic().clip(aoi)

print('Count of NAIP 13:', naip13.size().getInfo())
print('Count of NAIP 15:', count15)
print('Count of NAIP 17:', naip17.size().getInfo())
#print('Count 19:', naip19.size().getInfo())

Count of NAIP 13: 101760
Count of NAIP 15: 119659
Count of NAIP 17: 121728


## Calculate NDVI for all three years of NAIP

In [46]:
#nir, r = imgs.select('N'), imgs.select('R')
ndvi15 = np15.normalizedDifference(["N", "R"])
ndvi13 = np13.normalizedDifference(["N", "R"])
ndvi17 = np17.normalizedDifference(["N", "R"])

ndvi = ndvi15

## Image Segmentatioin and Feature Extraction with NAIP Images

### Use most two recent NAIP images (NAIP 2015 and 2017) for image segmentation

In [47]:
np_all = np15.addBands(np17.select('R','G','B','N'))
np_all.bandNames().getInfo()

['R', 'G', 'B', 'N', 'R_1', 'G_1', 'B_1', 'N_1']

In [48]:
seed = ee.Algorithms.Image.Segmentation.seedGrid(6)
#seg = ee.Algorithms.Image.Segmentation.GMeans(image=imgs,numIterations=100,pValue=50,neighborhoodSize=500)
#seg = ee.Algorithms.Image.Segmentation.SNIC(image=np15, size=10,compactness= 0, neighborhoodSize=500,connectivity= 8, seeds=seed).select(['R_mean', 'G_mean', 'B_mean', 'N_mean', 'clusters'], ['R', 'G', 'B', 'N', 'clusters'])
seg = ee.Algorithms.Image.Segmentation.KMeans(np_all, 10, 100, 10)
clusters = seg.select('clusters')

### Calculate ndvi and area of each segment/object, and filter segments by NDVI and area

In [49]:
seg_nv = ndvi17.addBands(clusters).reduceConnectedComponents(ee.Reducer.mean(), 'clusters').rename('seg_nv') # calculate segments ndvi
clusters_veg = clusters.updateMask(seg_nv.gte(0.1)) # keep segments with NDVI greater than or equal to(gte) 0.1
area = ee.Image.pixelArea().addBands(clusters).reduceConnectedComponents(ee.Reducer.sum(), 'clusters') # calculate segments area
clusters_veg = clusters_veg.updateMask(area.lte(200)).updateMask(area.gte(5)) # keep segments with area between 5-200 

clusters_AllVeg =  clusters.updateMask(seg_nv.gte(0.1)) # for another purpose, only filter segs by ndvi

### Calculate features for each segment, including mean band value, standard deviation of nir and ndvi, and textural features¶

In [50]:
fea_imgs = ee.Image([np13, np15, np17, ndvi13, ndvi15, ndvi17])
fea_imgs2 = ee.Image([np13.select("N"),np15.select("N"),np17.select("N"), ndvi13, ndvi15, ndvi17])

cluster2=clusters

## mean of all bands and ndvi
npmeans = fea_imgs.addBands(cluster2).reduceConnectedComponents(ee.Reducer.mean(),'clusters')

##  standard deviation of nir and ndvi
npstds =  fea_imgs2.addBands(cluster2).reduceConnectedComponents(ee.Reducer.stdDev(),'clusters')

## geometric features
## area
#area = ee.Image.pixelArea().addBands(cluster2).reduceConnectedComponents(ee.Reducer.sum(), 'clusters').rename('area')

## perimeter
#minMax = cluster2.reduceNeighborhood(ee.Reducer.minMax(), ee.Kernel.square(1))
#perimeterPixels = minMax.select(0).neq(minMax.select(1)).rename('perimeter')
#perimeter = perimeterPixels.addBands(cluster2).reduceConnectedComponents(ee.Reducer.sum(), 'clusters').rename('perimeter')

## width and height
#sizes = ee.Image.pixelLonLat().addBands(cluster2).reduceConnectedComponents(ee.Reducer.minMax(), 'clusters')
#width = sizes.select('longitude_max').subtract(sizes.select('longitude_min')).rename('width')
#height = sizes.select('latitude_max').subtract(sizes.select('latitude_min')).rename('height')


## textural features
glcm13=np13.select('N').glcmTexture(size= 3)
glcm15=np15.select('N').glcmTexture(size= 3)
glcm17=np17.select('N').glcmTexture(size= 3)

glcm_sele=['N_contrast','N_var','N_ent','N_savg','N_diss']
txtr_sele13=glcm13.select(glcm_sele)
txtr_sele15=glcm15.select(glcm_sele)
txtr_sele17=glcm17.select(glcm_sele)

texture_fea = ee.Image([txtr_sele13,txtr_sele15,txtr_sele17])

texture = texture_fea.addBands(cluster2).reduceConnectedComponents(ee.Reducer.mean(),'clusters')

## compile all feature images
#all_fea = ee.Image.cat([npmeans, npstds, area, perimeter, width, height, texture])
all_fea = ee.Image.cat([npmeans, npstds, area, texture])
print(all_fea.bandNames().getInfo())
print ("Number of variable is: " + str(len(all_fea.bandNames().getInfo())))

['R', 'G', 'B', 'N', 'R_1', 'G_1', 'B_1', 'N_1', 'R_2', 'G_2', 'B_2', 'N_2', 'nd', 'nd_1', 'nd_2', 'N_3', 'N_1_1', 'N_2_1', 'nd_3', 'nd_1_1', 'nd_2_1', 'area', 'N_contrast', 'N_var', 'N_ent', 'N_savg', 'N_diss', 'N_contrast_1', 'N_var_1', 'N_ent_1', 'N_savg_1', 'N_diss_1', 'N_contrast_2', 'N_var_2', 'N_ent_2', 'N_savg_2', 'N_diss_2']
Number of variable is: 37


## Load Host Presence Points (here use honeysuckle (HS) as a case)

In [51]:
os.chdir('/content/drive/Shared drives/APHIS  Projects/eRADS/eRADS_workflow/eRADS_workflow/host_mapping')

In [52]:
hs = pd.read_csv("ECFF_HS.csv",header=0)
hs = hs.dropna()
hs = hs[(hs.Longitude < long_max) & (hs.Longitude > long_min)]
hs = hs[(hs.Latitude < lat_max) & (hs.Latitude > lat_min)]

print(hs.head(5))
print("Dimension of coordinates data is: {}".format(hs.shape))

      Longitude  Latitude
122   -79.16519  43.80910
325   -79.69567  43.71885
667   -79.56413  44.14809
2498  -77.80124  44.28991
2527  -77.79989  44.24636
Dimension of coordinates data is: (427, 2)


### Convert point dataframe to earth engine point list

In [53]:
crdsls = hs.values.tolist()
crdsls = ee.List(crdsls)
#pts = ee.Geometry.MultiPoint(crdsls, proj='EPSG:4326')
hs_pts = ee.Geometry.MultiPoint(crdsls)
#pts = ee.FeatureCollection(pts)
hs_pts = ee.Feature(hs_pts)

ctd_long, ctd_lat = hs['Longitude'].mean(), hs['Latitude'].mean()
print("Centroid of all presence points is:{} {}".format(ctd_long, ctd_lat))

Centroid of all presence points is:-79.47584278688524 43.629590913348856


## Generate samples for each class (only have 2 classes here, 1 for HS and 1 for non-HS ) for classification

A critical assumption here is that: vegetations which are within 15-m of the presence points are the target host, i.e. HS in this case, whereas vegetations whoes distance to the presence points are between 15-100 are not target host species. Thus the vegetations within 15-m of the presence points are taken as the host species, whereas vegetations whose distance to presence points is between 15-100 are taken as non-host species. The other more practical interpretation is that the vegetations which are within 15-m of the presence points are more likely have similar features with locations where HS is presence. Thus this approach aims at picking up vegetations which have similar features with locations where HS is highly present. You can change the two distance, 15 and 100, based on specific host. 

### Create buffer for each sample points¶

In [54]:
hs_bf0 = hs_pts.buffer(100)
hs_bf1 = hs_pts.buffer(15)
hs_bfdf = hs_bf0.difference(hs_bf1)
hs_bfdf

hs_pre = hs_bf1
hs_no = hs_bfdf

### Clip image segments by buffers

In [55]:
clusters_hs = clusters_AllVeg.clip(hs_pre)
clusters_nonhs = clusters_AllVeg.clip(hs_no)

### Generate two classes, pre_img (HS presence - class 2) and no_img (HS absence - class 1), the output "samples" is an image with either value 1 or 2

In [56]:
pre_img = clusters_hs.select('clusters').multiply(0).add(2).rename('class')
no_img = clusters_nonhs.select('clusters').multiply(0).add(1).rename('class')
pre_img = pre_img.clip(aoi).unmask()
no_img = no_img.clip(aoi).unmask()
#samples = ee.ImageCollection.fromImages([pre_img, no_img]).mosaic().rename('class')
samples = pre_img.add(no_img)
samples = samples.updateMask(samples.gt(0))

## Select Training Data

### Confine the training area

In [57]:
train_aoi = hs_pre.union(hs_no)
feas_train = all_fea.clip(train_aoi)
#feas_train = all_fea.updateMask(samples.gte(1))

feas_train2 = feas_train.addBands(samples)
print(feas_train2.bandNames().getInfo())

train_fc = feas_train.reduceToVectors(scale=1, bestEffort = True, geometryType = 'polygon',maxPixels = 1e13, labelProperty = "class")
train_fc2 = train_fc.randomColumn('random')

train_sp = train_fc2.filter(ee.Filter.gte('random', 0.3))
test_sp = train_fc2.filter(ee.Filter.lt('random', 0.3))

['R', 'G', 'B', 'N', 'R_1', 'G_1', 'B_1', 'N_1', 'R_2', 'G_2', 'B_2', 'N_2', 'nd', 'nd_1', 'nd_2', 'N_3', 'N_1_1', 'N_2_1', 'nd_3', 'nd_1_1', 'nd_2_1', 'area', 'N_contrast', 'N_var', 'N_ent', 'N_savg', 'N_diss', 'N_contrast_1', 'N_var_1', 'N_ent_1', 'N_savg_1', 'N_diss_1', 'N_contrast_2', 'N_var_2', 'N_ent_2', 'N_savg_2', 'N_diss_2', 'class']


### Within the training area, sample training and testing data

In [68]:
training = feas_train2.stratifiedSample(1000, classBand= 'class',scale = 5)
testing = feas_train2.stratifiedSample(1000, classBand= 'class',scale = 5)

## Train Classifier

### Train classification model, gee has multiple models to choose¶. Here use random forest

In [69]:
classifier = ee.Classifier.randomForest(numberOfTrees = 256).train(training, 'class')
test = testing.classify(classifier)
ts_matrix = test.errorMatrix('class', 'classification')
classified = training.classify(classifier)
classified

### Check accuraccuracy (this step can take a long time) 

In [70]:
confMatrix = classifier.confusionMatrix()

OA = confMatrix.accuracy()
CA = confMatrix.consumersAccuracy()
PA = confMatrix.producersAccuracy()
Kappa = confMatrix.kappa()
Order = confMatrix.order()

In [72]:
# This step also can take a long time #

#print('Resubstitution error matrix: ', confMatrix.getInfo())
#print('Training overall accuracy: ', OA.getInfo())
#print('User accuracy: ', CA.getInfo())
#print('Producer accuracy: ', PA.getInfo())
#print('Kappa: ', Kappa.getInfo())

## Classify the Whole Study Area Using the Trained Classifier and Export Host Class

### Use the trained model to classify all area

In [28]:
classified_map = all_fea.classify(classifier)
host_map = classified_map.updateMask(classified_map.eq(2)) #(class 2 is the class for HS)

### Export the classified HS Map  (This step can take a long time)

In [ ]:
task = ee.batch.Export.image.toDrive(image= cla2, maxPixels = 1e13, description='HS_map',scale=100,folder='host_mapping',fileFormat='GeoTIFF')
task.start()
task.status()['state']

import time
starttime=time.time()
state = "Ready"  

n=0
while state!='COMPLETED' and state!= 'FAILED':
    print("status: " + task.status()['state'])
    state = task.status()['state']
    print("Time consumed: " + str(10*n) +'s')
    n += 1
    time.sleep(10)

if task.status()['state'] == 'FAILED':
    print (task.status()['error_message'])